In [2]:
import math
from math import factorial
from math import exp
import scipy
from random import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.distributions as tdist


In [3]:
import os
import sys
import inspect
    
# insert root dir into sys
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
root_dir = os.path.dirname(currentdir)
# print(root_dir)

if root_dir not in sys.path:
     sys.path.insert(0, root_dir)


##### Acceptance-rejection alg
To find $f(x) = C h(x) g(x)$<br>
where $C \geq 1$, `h(x)` is pdf, $g(x)\leq 1$<br>
([1. p. 45])<br><br>
1) Generate `U` from `Uniform(0, 1)` <br>
2) Generate `Y` from pdf `h(y)` <br>
3) if `U` $\leq$ `g(Y)` use `Y` as `X` (i.e. `X=Y` - accept)<br>
else reject <br>
go to 1)

Prove [1. p 46]:<br>
$f(x) = C h(x) g(x)$<br>
=><br>
$P(Y=x|U\leq g(Y)) = f_{Y}(x|U\leq g(Y))=f_{X}(x)$


Von Neumann implementation [1. p. 47]:<br>
1) Generate $U_{1}$ and $U_{2}$ from `Uniform(0, 1)` <br>
2) $Y \leftarrow a+ U_{2}(b-a)$  (i.e. $h(x)=\frac{1}{b-a}$)<br>
3) if $U_{1}\leq g(Y) = \frac{f_{X}(Y)}{M}$  use `Y` as `X` (i.e. `X=Y` - accept) else reject <br>
go to 1)

In [33]:
class ARDist():
    '''Acceptance-rejection alg (p.47 in [1.])'''
    
    def __init__(self, a, b, f):
        self.a = a
        self.b = b
        self.f = f
        
        self.usampler = lambda n: np.random.random(n)
        
        # hdist = UniformAB1(a, b)
        # self.hsampler = hdist.sample_n
        inv_H = lambda u: a+(b-a)*u 
        self.hsampler = lambda n: inv_H(np.random.random(n))
        
        self.M = f(np.random.uniform(a, b, 10000)).max()
        self.C = self.M*(b-a)
        
    def sample_n(self, n):
        M = self.M
        f = self.f
        u1 = self.usampler(n)
        y = self.hsampler(n)
        return(y[u1 <= f(y)/M])

In [56]:
# f = lambda x: 7*np.sin(x)+np.sin(4*x)
f = lambda x: 7*np.sin(x)+np.sin(14*x)

ar = ARDist(0, np.pi, f)
print(ar.M)
print(ar.C)
print(ar.usampler(3))
print(ar.hsampler(3))
len(ar.sample_n(1700))

7.957498813668379
24.999219813970075
[0.42895434 0.67539231 0.7236296 ]
[0.55694826 0.96694752 2.33599492]


943

In [57]:
%matplotlib
xs = ar.sample_n(17000)
m1 = max(xs)
# xs = ar.hsampler(17000)
fig, ax = plt.subplots()

v, b, p = ax.hist(xs, 70, # normed=True, # density=True, 
                stacked=True)
xs1 = np.linspace(0, np.pi, 17000)
ax.plot(xs1, (f(xs1)/ar.M)*max(v))
# plt.hist(xs, 70, # density=True, 
#         stacked=True)


Using matplotlib backend: Qt5Agg


### Refs:
1) Simulation and the Monte Carlo Method by Reuven Y. Rubinstein